# Assistant-11<br/>Travel Assistant

### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import oaihelper as helper

## Get an OpenAI client
client = helper.get_openai_client(api_key=helper.api_KEY,
        api_version=helper.api_version,
        azure_endpoint=helper.api_URI,)

### Load the files

In [2]:
## Files
file_itinerary = helper.upload_file(client,"../data/travel/flight_itinerary.txt")
file_bills = helper.upload_file(client,"../data/travel/airbnb_bills.csv")
#file_trains = helper.upload_file(client,"../data/train_ids.csv")
#file_stations = helper.upload_file(client,"../data/stations.csv")

Added file:  assistant-gVXZDFW7a9cqTMZN4M6PGIMB 1
Added file:  assistant-tZo4u2vH1dt6XwaTaJ75q0ep 2


In [3]:
def mock_bloc_calendar(subject,start,end):
    print(f"Subject: {subject} \nStart: {start} \nEnd: {end}")

### Define the tools to use in the Assistants

In [4]:
tools_list = [
    {"type":"retrieval"},    
    {"type":"code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "block_calendar",
            "description": "Sends a calendar event.",
            "parameters": {
                "type": "object",
                "properties": {
                    "eventSubject": {
                        "type": "string",
                        "description": "Event description."
                    },
                    "startTime": {
                        "type": "string",
                        "description": "Event start time and date."
                    }
                    ,
                    "endTime": {
                        "type": "string",
                        "description": "Event end time and date."
                    }
                },
                "required": ["eventSubject", "startTime","endTime"]
            }
        }
    }
]

### Define the Assistant

In [5]:
helper.clear_shelves()

travel_bot_assistant = helper.create_assistant(client,
        name="Travel Assistant",
        instructions="You are a helpful, friendly, and professional assistant that can help with travel plans, calculate expenses, and answer questions based on the provided knowledge.",        
        tools=tools_list,
        model=helper.gpt_deployment_name,
        file_ids=[file_itinerary.id,file_bills.id],)

Added assistant:  asst_qLpRyEt4DGtVsFW8TY8y6ift 1


### Define the delegate to handle function calling

In [6]:
def function_calling_delegate(client, thread, run):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "block_calendar":
            print("blocking calendar...")
            subject = arguments['eventSubject']
            start = arguments['startTime']
            end = arguments['endTime']
            mock_bloc_calendar(subject,start,end)            
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": "Calendar blocked"
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Process user messages

In [7]:
helper.generate_response(client,travel_bot_assistant,"Suggest a day-by-day itinerary for the dates of my travel? I like walk around the city I'm visiting and go on day trips?", "123", "Alex",function_calling_delegate)
helper.generate_response(client,travel_bot_assistant,"Will be 4 friends staying at this Airbnb in Berlin. What's my share of costs+my flight?", "123", "Alex",function_calling_delegate)
helper.generate_response(client,travel_bot_assistant,"Block my calendar for a week starting today.", "123", "Alex",function_calling_delegate)


Creating new thread for Alex with user_id 123
Added thread:  thread_k24BmB5QT3AZPZU2Sth1K99h 1
Generated message: Based on your travel itinerary, you will be departing from Miami International Airport (MIA) to Frankfurt Airport (FRA) on September 28, 2024, and returning on October 12, 2024. That gives you about two weeks in and around Frankfurt, Germany. 

Being interested in walking around the city and day trips, here's a suggested day-by-day itinerary for your stay:

### September 29, 2024 - Arrival in Frankfurt
- **Morning:** Rest after your flight and get settled in your accommodation.
- **Afternoon:** Explore the area around your hotel and familiarize yourself with the city. Perhaps a light walk through the city center.
- **Evening:** Have a relaxing dinner nearby and rest up for the next day.

### September 30, 2024 - Frankfurt City Tour
- **Morning:** Visit the iconic Römerberg, Frankfurt's old central square.
- **Afternoon:** Explore Frankfurt Cathedral and enjoy the panoramic 

'Your calendar has been blocked for a week starting today, from January 16, 2024, to January 23, 2024.'

### Delete all demo objects

In [8]:
helper.cleanup(client)

Deleting:  1  assistants.
AssistantDeleted(id='asst_qLpRyEt4DGtVsFW8TY8y6ift', deleted=True, object='assistant.deleted')
Deleting:  1  threads.
ThreadDeleted(id='thread_k24BmB5QT3AZPZU2Sth1K99h', deleted=True, object='thread.deleted')
Deleting:  2  files.
FileDeleted(id='assistant-gVXZDFW7a9cqTMZN4M6PGIMB', deleted=True, object='file')
FileDeleted(id='assistant-tZo4u2vH1dt6XwaTaJ75q0ep', deleted=True, object='file')
